# Data Leakage

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, make_scorer

## Learning Goals

- avoid letting information about test sets get into the training of models
- use best practices for building non-leaky workflows
- repair leaky workflows

## Introduction

We have encountered the idea of splitting our data into two, *training* our model on one bit and then *testing* it on the other.

The goal is to have an unbiased assessment of our model, and so we want to make sure that nothing about our test data sneaks into the training run of the model.

## A Mistake

Now consider the following workflow:

In [ ]:
data = load_diabetes()

print(data.DESCR)

df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

In [ ]:
X, y = load_diabetes(return_X_y=True)

Scale and run a regression


In [ ]:
# Code

Well we fit the model only to our training data. Looks like we've done everything right, right?

It's important to understand that the answer here is a resounding "NO". It's true that we didn't directly fit our model to our test data. But the trouble is that we fit our scaler **to the whole dataset**. That is, records in the test data are contributing to calculations of column means and standard deviations, and so, surreptitiously, information about our test set is sneaking into the training run of the model after all!

To correct our mistake, we'll make sure to perform our train-test split **first**:

In [ ]:
# Code

Note that our model coefficients are slightly different from what they were before.

### Error Comparison

It's worth pointing out that, **for linear models**, there is **no** difference in modeling error:

In [ ]:
y_test_hat = lr.predict(X_test)
mse = mean_squared_error(y_test, y_test_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

In [ ]:
y_test2_hat = lr2.predict(X_test2_scld)
mse = mean_squared_error(y_test2, y_test2_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

This will **not** be true for other sorts of models that use different loss functions.

## Preprocessing

In general all preprocessing steps are subject to the same dangers here. Consider the preprocessing step of one-hot-encoding:

In [ ]:
gun_poll = pd.read_csv('data/guns-polls.csv')

In [ ]:
gun_poll.head()

In [ ]:
gun_poll['Pollster'].value_counts()

Now if I were to fit a one-hot encoder to the whole `Pollster` column here, the encoder would learn all the categories. But I need to prepare myself for the real-world possibility that unfamiliar categories may show up in future records. Let's explore this.

In [ ]:
# First I'll do a split


Let's suppose now that I fit a `OneHotEncoder` to the `Pollster` column in my training data.

#### Exercise

Fit an encoder to the `Pollster` column of the training data and then check to see which categories are represented.

In [ ]:
# Code

In [ ]:
# So what categories do we have?

We'll want to transform both train and test after we've fitted the encoder to the train.

In [ ]:
test_to_be_dummied = X_test[['Pollster']]

ohe.transform(to_be_dummied)
ohe.transform(test_to_be_dummied)

There are categories in the testing data that don't appear in the training data! What should 
we do about that?

### Approaches

- **Strategy 1**: Divide up the categories proportionally when we do our train_test_split. If we're using `sklearn`'s tool, that means taking advantage of the `stratify` parameter:

In [ ]:
new_X_train, new_X_test = train_test_split(gun_poll,
                                           stratify=gun_poll['Pollster'],
                                           random_state=42)

Unfortunately, in this case, we can't use this since some categories have only a single member.

- **Strategy 2**: Drop the categories with very few representatives.

In the present case, let's try dropping the single-member categories.

In [ ]:
# Code

In [ ]:
bad_cols

In [ ]:
# Remove bad columns

In [ ]:
# Sniff test!
gun_poll['Pollster'].value_counts()

We could now split this carefully so that new categories don't show up in the testing data. In fact, now we can try the stratified split:

In [ ]:
X_train3, X_test3 = train_test_split(gun_poll,
                                     stratify=gun_poll['Pollster'],
                                     test_size=0.3,
                                     random_state=42)

In [ ]:
# Sniff Test

Now every category that appears in the test data appears also in the training data.

- **Strategy 3**: Adjust the settings on the one-hot-encoder.

For `sklearn`'s tool, we'll tweak the `handle_unknown` parameter:

#### Exericse

Fit a new encoder to our training data column that won't break when we try to use it to transform the test data. And then use the encoder to transform both train and test.

In [ ]:
ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(to_be_dummied)
test_to_be_dummied = X_test[['Pollster']]
ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)

## Leakage into Validation Data

If we employ cross-validation, then our training data points will be serving both for training and for validation. So there's a sense in which we can't help but let some information about our validation data sneak into the model.

But strictly speaking, cross-validation means building *multiple* models, and we still want each to be blind to its validation set.

The dangers of data leakage, therefore, are still very much real in the case of validation data. And they are often more subtle as well. Consider the following line of code:

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Using our scaled training data

cv_results = cross_validate(estimator=LinearRegression(),
                X=X_train2_scld,
                y=y_train2,
                scoring=mse,
                return_estimator=True)

In [ ]:
# Looking at model coefficients on the first predictor

[model.coef_[0] for model in cv_results['estimator']]

We've built five models here, and none of them saw any points from the test data, so we have no leaks, right?

Wrong! We fit the `StandardScaler` to the whole training set, which means that information about *every* fold will affect every cross-validation. A better practice here would be to split our data into its cross-validation folds *first*. Then we can fit the scaler to only the training folds for each cross-validation.

Of course, the more preprocessing steps we have, the more tedious it becomes to do this work! For such tasks it is often greatly beneficial to take advantage of `sklearn`'s `Pipeline`s, which we'll have more to say about later.

For now, let's see if we can fix our leaky cross-validation scorer.

#### Exercise

Go back to `X_train2` and do the following:

- Split it into five validation folds (Use `KFold()`).
- For each split:
- (i) fit a `StandardScaler` to the four-fold chunk and transform all data points with it.
- (ii) fit a `LinearRegression` to the four-fold chunk and print out the value of the first coefficient.

In [ ]:
# Code

### A Contrived But Illustrative Example

This is only a slight difference in coefficients, but for other datasets the difference can be great.

In [ ]:
fake_preds = np.array([6, 5, 2, 250, 300]).reshape(-1, 1)
fake_target = np.array([25, 30, 12, 400, 420]).reshape(-1, 1)

In [ ]:
df = pd.DataFrame(np.hstack((fake_preds, fake_target)),
                 columns=['pred', 'target'])

In [ ]:
df

In [ ]:
small_train = df[['pred']][:3]
small_val = df[['pred']][3:]
small_train_y = df['target'][:3]
small_val_y = df['target'][3:]

In [ ]:
# Scaling the whole dataset

ss = StandardScaler().fit(df[['pred']])

In [ ]:
X_tr = ss.transform(small_train)
X_va = ss.transform(small_val)

In [ ]:
lr = LinearRegression().fit(X_tr, small_train_y)
lr.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr.coef_ * X + lr.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling the whole dataset is
    {round(lr.coef_[0])}X + {round(lr.intercept_)}""");

In [ ]:
# Splitting before scaling

ss2 = StandardScaler().fit(small_train)

In [ ]:
X_tr2 = ss2.transform(small_train)
X_va2 = ss2.transform(small_val)

In [ ]:
lr2 = LinearRegression().fit(X_tr2, small_train_y)
lr2.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr2.coef_ * X + lr2.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling after splitting is
    {round(lr2.coef_[0])}X + {round(lr2.intercept_)}""");